This notebook can be executed to run all experiments using the darts library.

Training using the Transformer model is unreliable, sometimes returning nan with the same input. 

In [1]:
from darts import TimeSeries
from darts.models import NBEATSModel, NHiTSModel, TransformerModel, TSMixerModel
from darts.utils.losses import *
from darts.metrics import metrics as darts_metrics
from utils import data_handling, helpers
import torch
import numpy as np
import os
import pickle
import pandas as pd
import config
import copy
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning.callbacks.early_stopping import EarlyStopping


# Constants
DEVICE = [1]
IN_LEN = 96
OUT_LEN = 96
LOSS_FN = torch.nn.MSELoss()
LAYER_WIDTH = 256
NUM_STACKS = 4
NUM_BLOCKS = 2
NUM_LAYERS = 3
COEFFS_DIM = 5
DROPOUT = 0.25
VERBOSE = True
TRAIN_EPOCHS = 15
TUNE_EPOCHS = 5
four_weeks = -24*7*4
LR = 0.005

metrics_output_path = config.CONFIG_OUTPUT_PATH["darts"] / "darts_metrics.csv"

model_path = config.CONFIG_MODEL_LOCATION["darts"]

/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/statsforecast/core.py:26: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
def extend_source_to_target_id_count(source, target):
    source_id_count = source["train"].shape[1]
    target_id_count = target["train"].shape[1]

    full_repeats = target_id_count // source_id_count
    remainder = target_id_count % source_id_count

    repeated_tensor = source["train"].repeat(1, full_repeats)
    remainder_tensor = source["train"][:, :remainder]
    source_train = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    
    assert target_id_count == source_train.size(1), f"Reshaping was incorrect. Target_train = {target_id_count}, source_train = {source_train.size(1)}."

    repeated_tensor = source["validation"].repeat(1, full_repeats)
    remainder_tensor = source["validation"][:, :remainder]
    source_validation = torch.cat((repeated_tensor, remainder_tensor), dim=1)
    assert target_id_count == source_validation.size(1), f"Reshaping was incorrect. Target_val = {target_id_count}, source_val = {source_validation.size(1)}."

    return source_train, source_validation


def process_tl_data(source_data, target_data):
    # either reshape source or target dataset according to which has less IDs
    source_ids = source_data["train"].size(1)
    target_ids = target_data["test"].size(1)

    fine_tune_horizon = -24*7*4
    target_test = target_data["test"]
    target_fine_tuning = target_data["train"][fine_tune_horizon:,:]

    # remove IDs if source is bigger than target or
    # repeat IDs if target is bigger than source
    if target_ids < source_ids:
        source_train = source_data["train"][:,:target_ids]
        source_validation = source_data["validation"][:,:target_ids]
    else:
        source_train, source_validation = extend_source_to_target_id_count(source_data, target_data)

    # convert to TimeSeries dataframe
    source_train = TimeSeries.from_values(source_train)
    source_validation = TimeSeries.from_values(source_validation)
    target_test = TimeSeries.from_values(target_test)
    target_fine_tuning = TimeSeries.from_values(target_fine_tuning)
    target_train = TimeSeries.from_values(target_data["train"])
    target_validation = TimeSeries.from_values(target_data["validation"])

    tl_dataset = {
                    "source_train" : source_train,
                    "source_validation" : source_validation,
                    "target_fine_tuning" : target_fine_tuning,
                    "target_test" : target_test,
                    "target_train" : target_train,
                    "target_validation" : target_validation
                }

    return tl_dataset

In [3]:
def train_nhits(ts_train, ts_val, epochs=1):   
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }

    nhits_model = NHiTSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        activation='ReLU',
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        dropout=DROPOUT,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        optimizer_kwargs={"lr": LR},
        pl_trainer_kwargs=TRAINER_ARGS,
    )

    nhits_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nhits_model

# Train NBEATS model
def train_nbeats(ts_train, ts_val, epochs=1):   
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }

    nbeats_model = NBEATSModel(
        input_chunk_length=IN_LEN,
        output_chunk_length=OUT_LEN,
        batch_size=32,
        num_stacks=NUM_STACKS,
        num_blocks=NUM_BLOCKS,
        num_layers=NUM_LAYERS,
        layer_widths=LAYER_WIDTH,
        expansion_coefficient_dim=COEFFS_DIM,
        loss_fn=LOSS_FN,
        use_reversible_instance_norm=True,
        activation='ReLU',
        optimizer_kwargs={"lr": LR},
        pl_trainer_kwargs=TRAINER_ARGS,
    )    

    nbeats_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return nbeats_model

# Train Transformer model
def train_transformer(ts_train, ts_val, epochs=1):    
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }

    transformer_model = TransformerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN,
        d_model=LAYER_WIDTH, 
        nhead=4, 
        num_encoder_layers=2, 
        num_decoder_layers=3, 
        dim_feedforward=LAYER_WIDTH, 
        dropout=DROPOUT, 
        activation='relu', 
        loss_fn=LOSS_FN,
        optimizer_kwargs={"lr": LR},
        use_reversible_instance_norm=True,
        pl_trainer_kwargs=TRAINER_ARGS,
        
        )
    
    transformer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return transformer_model 

# Train TSMixer model
def train_tsmixer(ts_train, ts_val, epochs=1):    
    TRAINER_ARGS = {"enable_progress_bar": True, 
                "accelerator": "gpu",  
                "devices" : DEVICE,
             }
    
    tsmixer_model = TSMixerModel(
        input_chunk_length=IN_LEN, 
        output_chunk_length=OUT_LEN, 
        hidden_size=LAYER_WIDTH, 
        ff_size=LAYER_WIDTH, 
        num_blocks=NUM_BLOCKS, 
        activation='ReLU', 
        dropout=DROPOUT, 
        loss_fn=LOSS_FN,
        norm_type='LayerNorm', 
        optimizer_kwargs={"lr": LR},
        use_reversible_instance_norm=True,
        pl_trainer_kwargs=TRAINER_ARGS,
    )

    tsmixer_model.fit(ts_train, val_series=ts_val, epochs=epochs, verbose=VERBOSE)
    return tsmixer_model

In [4]:
def evaluate(model, target_test):
    """
    Evaluates models on target test set
    Input:  -trained model
            -List of target test sets shaped according to models

    Output: Dict{MSE, MAE}
    """

    
    # check for last input point and create input/target lists of 96 horizons
    forecasting_endpoint = int(len(target_test)) - 96*2
    window = [target_test[i:i+96] for i in range(0, forecasting_endpoint, 5)]
    target = [target_test[i+96:i+96+96] for i in range(0, forecasting_endpoint, 5)]

    # predict over dataloader with slidingwindow implementation and 5 time step shifts for each input
    predictions = model.predict(n=96, series=window)

    mse = darts_metrics.mse(predictions, target)
    mae = darts_metrics.mae(predictions, target)

    mse = sum(mse) / len(predictions)
    mae = sum(mae) / len(predictions)

    return {'MSE': mse, 'MAE': mae}


def fine_tune_model(model, target_fine_tuning, epochs=1):
    """
    Fine tune models over specified epochs

    Input:  -trained models
            -fine tuning dataset
            -epochs

    Returns: fitted models
    """

    
    model.fit(
            target_fine_tuning,
            num_loader_workers=4,
            epochs=epochs,
            max_samples_per_ts=None,
        )
 
    return model

In [5]:
# use electricity dataset
electricity_dict = data_handling.format_electricity()

for key, value in electricity_dict.items():
			electricity_dict[key]= data_handling.df_to_tensor(value)

# normalize train and use matrics for val and test
electricity_dict["4_weeks_train"] = electricity_dict["train"][four_weeks:,:]
electricity_dict["train"], train_standardize_dict = helpers.custom_standardizer(electricity_dict["train"])
electricity_dict["validation"], _ = helpers.custom_standardizer(electricity_dict["validation"], train_standardize_dict)
electricity_dict["test"], _ = helpers.custom_standardizer(electricity_dict["test"], train_standardize_dict)
electricity_dict["4_weeks_train"], _ = helpers.custom_standardizer(electricity_dict["4_weeks_train"], train_standardize_dict)

# bavaria dataset
data_tensor = data_handling.load_bavaria_electricity()
bavaria_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
bavaria_dict["4_weeks_train"] = bavaria_dict["train"][four_weeks:,:]

# building genome project dataset
data_tensor = data_handling.load_genome_project_data()
gp_dict, standadizer = data_handling.train_test_split_eu_elec(data_tensor, standardize=True)
gp_dict["4_weeks_train"] = gp_dict["train"][four_weeks:,:]

Length train set: 209 days, 0:00:00
Length validation set: 34 days, 0:00:00
Saving train, validation and test df for faster loading


In [6]:
tl_setups = {
    "ELD_to_Bavaria" : (electricity_dict, bavaria_dict), 
    "ELD_to_GP2" : (electricity_dict, gp_dict),
    "Bavaria_to_ELD" : (bavaria_dict, electricity_dict), 
    "Bavaria_to_GP2" : (bavaria_dict, gp_dict), 
    "GP2_to_Bavaria": (gp_dict, bavaria_dict), 
    "GP2_to_ELD" : (gp_dict, electricity_dict)
     }

In [8]:
models = {
    "NHiTS": train_nhits,
  #  "NBEATS": train_nbeats,
    "Transformer": train_transformer,
    "TSMixer": train_tsmixer
}

# Initialize the DataFrame
try:
    results_df = pd.read_csv(metrics_output_path, index_col=[0, 1, 2])
except FileNotFoundError:
    metrics = ["MSE", "MAE"]
    learning_scenarios = ["Zero-Shot", "four_weeks_tl", "full_tl", "full_training", "four_weeks_training"]
    index = pd.MultiIndex.from_product([tl_setups.keys(), learning_scenarios, metrics], names=["Setup", "Learning_scenario", "Metric"])
    results_df = pd.DataFrame(columns=models.keys(), index=index)

# Helper functions
def update_metrics(setup_name, model_name, learning_scenario, mae, mse):
    results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
    results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse

def is_metric_filled(setup_name, model_name, learning_scenario):
    # Check if specific metrics for a model in a setup and fine-tuning scenario are NaN or not
    metrics_filled = not results_df.loc[(setup_name, learning_scenario, slice(None)), model_name].isnull().any()
    return metrics_filled

def run_model_tl(setup_name, tl_data):
    source_train = tl_data["source_train"]
    source_val = tl_data["source_validation"]
    target_fine_tuning = tl_data["target_fine_tuning"]
    target_test = tl_data["target_test"]
    target_train = tl_data["target_train"]
    #target_val = tl_data["target_validation"]

    model_trained = False
    for model_name, model_func in models.items():
        if is_metric_filled(setup_name, model_name, "Zero-Shot") :
            print(f"Skipping {model_name} for {setup_name} with zero-shot metrics already filled.")
        else:          
            # Train model with source dataset (Zero-Shot)
            model = model_func(source_train, source_val, epochs=TRAIN_EPOCHS)
            metrics = evaluate(model, target_test)
            model_trained = True
            update_metrics(setup_name, model_name, "Zero-Shot", metrics['MAE'], metrics['MSE'])
            full_tl_model = copy.deepcopy(model)
            print(f"Metrics updated for {model_name} in {setup_name} with zero-shot fine-tuning: MAE = {metrics['MAE']}, MSE = {metrics['MSE']}")
        
        results_df.to_csv(metrics_output_path)

        if is_metric_filled(setup_name, model_name, "four_weeks_tl") :
            print(f"Skipping {model_name} for {setup_name} with shot TL metrics already filled.")
        else:          
            # Fine-tune on small target train set (four_weeks_tl)
            if model_trained == False:
                model = model_func(source_train, source_val, epochs=TRAIN_EPOCHS)
            model = fine_tune_model(model, target_fine_tuning, epochs=TUNE_EPOCHS + 5)
            metrics = evaluate(model, target_test)
            update_metrics(setup_name, model_name, "four_weeks_tl", metrics['MAE'], metrics['MSE'])
            print(f"Metrics updated for {model_name} in {setup_name} with zero-shot fine-tuning: MAE = {metrics['MAE']}, MSE = {metrics['MSE']}")
        results_df.to_csv(metrics_output_path)


        if is_metric_filled(setup_name, model_name, "full_tl") :
            print(f"Skipping {model_name} for {setup_name} with full TL metrics already filled.")
        else:          
            # Fine-tune on full target train set (four_weeks_tl)
            if model_trained == False:
                full_tl_model = model_func(source_train, source_val, epochs=TRAIN_EPOCHS)
            # Fine-tune on full target train set (full_tl)
            full_tl_model = fine_tune_model(full_tl_model, target_train, epochs=TUNE_EPOCHS)
            metrics = evaluate(full_tl_model, target_test)
            update_metrics(setup_name, model_name, "full_tl", metrics['MAE'], metrics['MSE'])
            print(f"Metrics updated for {model_name} in {setup_name} with zero-shot fine-tuning: MAE = {metrics['MAE']}, MSE = {metrics['MSE']}")

        # Save after every dataset combination
        results_df.to_csv(metrics_output_path)

def train_baselines(setup_name, tl_data):
    target_fine_tuning = tl_data["target_fine_tuning"]
    target_test = tl_data["target_test"]
    target_train = tl_data["target_train"]
    target_validation = tl_data["target_validation"]
 
    for model_name, model_func in models.items():
        if is_metric_filled(setup_name, model_name, "four_weeks_training"):
            print(f"Skipping {model_name} for {setup_name} with as baseline metrics are already filled.")
        else:
             # Train on short target train set (four_weeks_training)
            model = model_func(target_fine_tuning, target_validation, epochs=TRAIN_EPOCHS)
            metrics = evaluate(model, target_test)
            update_metrics(setup_name, model_name, "four_weeks_training", metrics['MAE'], metrics['MSE'])

        results_df.to_csv(metrics_output_path)

        if is_metric_filled(setup_name, model_name, "full_training"):
            print(f"Skipping {model_name} for {setup_name} with as metrics are already filled.")
        else:
            # Train on full target train set (full_training)
            model = model_func(target_train, target_validation, epochs=15)
            metrics = evaluate(model, target_test)
            update_metrics(setup_name, model_name, "full_training", metrics['MAE'], metrics['MSE'])

        results_df.to_csv(metrics_output_path)

# Execute for each setup and fine-tuning scenario
for setup_name, (source_data, target_data) in tl_setups.items():
    print(setup_name)
    tl_data = process_tl_data(source_data, target_data)
    run_model_tl(setup_name, tl_data)
    train_baselines(setup_name, tl_data)

results_df

ELD_to_Bavaria
Skipping NHiTS for ELD_to_Bavaria with zero-shot metrics already filled.
Skipping NHiTS for ELD_to_Bavaria with shot TL metrics already filled.
Skipping NHiTS for ELD_to_Bavaria with full TL metrics already filled.
Skipping Transformer for ELD_to_Bavaria with zero-shot metrics already filled.
Skipping Transformer for ELD_to_Bavaria with shot TL metrics already filled.
Skipping Transformer for ELD_to_Bavaria with full TL metrics already filled.
Skipping TSMixer for ELD_to_Bavaria with zero-shot metrics already filled.
Skipping TSMixer for ELD_to_Bavaria with shot TL metrics already filled.
Skipping TSMixer for ELD_to_Bavaria with full TL metrics already filled.
Skipping NHiTS for ELD_to_Bavaria with as baseline metrics are already filled.
Skipping NHiTS for ELD_to_Bavaria with as metrics are already filled.
Skipping Transformer for ELD_to_Bavaria with as baseline metrics are already filled.
Skipping Transformer for ELD_to_Bavaria with as metrics are already filled.
Skippi

/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/py ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]

  | Name                | Type                | Params
------------------------------------------------------------
0 | criterion           | MSELoss             | 0     
1 | train_metrics       | MetricCollection    | 0     
2 | val_metrics         | MetricCollection    | 0     
3 | rin                 | RINorm              | 2.9 K 
4 | encoder             | Linear              | 372 K 
5 | positional_encoding | _PositionalEncoding |

Epoch 14: 100%|██████████| 304/304 [00:14<00:00, 20.36it/s, train_loss=0.00037, val_loss=nan.0]    

`Trainer.fit` stopped: `max_epochs=15` reached.


Epoch 14: 100%|██████████| 304/304 [00:14<00:00, 20.36it/s, train_loss=0.00037, val_loss=nan.0]


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2]


Predicting DataLoader 0: 100%|██████████| 18/18 [00:01<00:00, 14.16it/s]


/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/darts/metrics/metrics.py:1159: RuntimeWarning: Mean of empty slice
  return np.nanmean(
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/darts/metrics/metrics.py:242: RuntimeWarning: Mean of empty slice
  vals = np.expand_dims(component_reduction(vals, axis=COMP_AX), axis=COMP_AX)
/vol/fob-vol7/nebenf21/reinbene/bene/MA/myenv/lib/python3.10/site-packages/darts/metrics/metrics.py:783: RuntimeWarning: Mean of empty slice
  return np.nanmean(
/tmp/ipykernel_43380/3178925498.py:19: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MAE"), model_name] = mae
/tmp/ipykernel_43380/3178925498.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  results_df.loc[(setup_name, learning_scenario, "MSE"), model_name] = mse


Skipping TSMixer for Bavaria_to_GP2 with as baseline metrics are already filled.
Skipping TSMixer for Bavaria_to_GP2 with as metrics are already filled.
GP2_to_Bavaria
Skipping NHiTS for GP2_to_Bavaria with zero-shot metrics already filled.
Skipping NHiTS for GP2_to_Bavaria with shot TL metrics already filled.
Skipping NHiTS for GP2_to_Bavaria with full TL metrics already filled.
Skipping Transformer for GP2_to_Bavaria with zero-shot metrics already filled.
Skipping Transformer for GP2_to_Bavaria with shot TL metrics already filled.
Skipping Transformer for GP2_to_Bavaria with full TL metrics already filled.
Skipping TSMixer for GP2_to_Bavaria with zero-shot metrics already filled.
Skipping TSMixer for GP2_to_Bavaria with shot TL metrics already filled.
Skipping TSMixer for GP2_to_Bavaria with full TL metrics already filled.
Skipping NHiTS for GP2_to_Bavaria with as baseline metrics are already filled.
Skipping NHiTS for GP2_to_Bavaria with as metrics are already filled.
Skipping Trans

NHiTS  Transformer    TSMixer
Setup          Learning_scenario   Metric                                   
ELD_to_Bavaria Zero-Shot           MSE      0.001619     0.001612   0.002046
                                   MAE      0.029768     0.029673   0.034182
               four_weeks_tl       MSE      0.000470     0.000489   0.000521
                                   MAE      0.010753     0.010680   0.012290
               full_tl             MSE      0.000285     0.000327   0.000276
                                   MAE      0.009179     0.009851   0.008743
               full_training       MSE      0.000284     0.000318   0.000304
                                   MAE      0.008886     0.009337   0.009330
               four_weeks_training MSE      0.000355     0.000498   0.000504
                                   MAE      0.010746     0.011392   0.011714
ELD_to_GP2     Zero-Shot           MSE      0.927760     0.986503   1.233384
                                   MAE      0.616421     0.655510   0.695620
               four_weeks_tl       MSE      0.995074     1.041323   0.865992
                                   MAE      0.641355     0.664097   0.524999
               full_tl             MSE      0.980549     0.975606   0.657875
                                   MAE      0.653280     0.649784   0.433311
               full_training       MSE      0.980830     0.977230   0.674455
                                   MAE      0.649966     0.650806   0.441427
               four_weeks_training MSE      1.001582     1.048865   1.053140
                                   MAE      0.660117     0.665634   0.577975
Bavaria_to_ELD Zero-Shot           MSE     14.306585    10.764251  13.597932
                                   MAE      3.264933     2.907783   2.809807
               four_weeks_tl       MSE      0.686743     0.958401   0.928438
                                   MAE      0.643095     0.813941   0.802077
               full_tl             MSE      0.363720     0.948382   0.214651
                                   MAE      0.414757     0.809264   0.300557
               full_training       MSE      0.358858     0.947744   0.226847
                                   MAE      0.414961     0.809475   0.309050
               four_weeks_training MSE      0.434020     0.954644   0.480004
                                   MAE      0.474205     0.813598   0.485502
Bavaria_to_GP2 Zero-Shot           MSE      6.936576     7.596890   8.128555
                                   MAE      1.952257     2.046025   2.061725
               four_weeks_tl       MSE      1.029320     1.059521   0.866328
                                   MAE      0.661392     0.673235   0.532831
               full_tl             MSE      0.976055          NaN   0.644111
                                   MAE      0.648918          NaN   0.428923
               full_training       MSE      0.978798          NaN   0.680707
                                   MAE      0.651744          NaN   0.443016
               four_weeks_training MSE      0.969528     1.043119   1.066945
                                   MAE      0.640075     0.664053   0.584738
GP2_to_Bavaria Zero-Shot           MSE      0.001653     0.001689   0.001333
                                   MAE      0.030085     0.030479   0.026276
               four_weeks_tl       MSE      0.000389     0.000488   0.000482
                                   MAE      0.010296     0.010822   0.011400
               full_tl             MSE      0.000277     0.000319   0.000273
                                   MAE      0.008924     0.009349   0.008793
               full_training       MSE      0.000290     0.000320   0.000301
                                   MAE      0.009144     0.009454   0.009010
               four_weeks_training MSE      0.000339     0.000499   0.000496
                                   MAE      0.009837     0.011575   0.011267
GP2_to_ELD     Zero-Shot           MSE     